In [1]:
import pandas as pd
import torch
import random
import re
import logging

from pytorch_pretrained_bert import BertTokenizer, BertConfig, BertModel
from pytorch_pretrained_bert.modeling import BertPreTrainedModel, BertPreTrainingHeads
from torch.utils.data import TensorDataset, DataLoader, SequentialSampler
from pytorch_pretrained_bert.modeling import BertForNextSentencePrediction
from pytorch_pretrained_bert.tokenization import BertTokenizer
from torch import nn
from torch.nn import CrossEntropyLoss

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [2]:
logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger("bert")

In [3]:
device = torch.device('cpu')#torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
f = open('/home/shivam/Documents/ether_language_model/bert_hugging_pytorch/custom/inputs/engg/se_medium_nltk.txt','r')
paragraphs = f.read().split("\n\n")

In [5]:
window = 3

In [6]:
def cleanText(text):
        
    #text = text.replace('\n','')
    text = text.replace('\\n','')
    text = text.replace('\n\n','')
    text = text.replace('\\','')
    text = text.replace('\t', '')
    #text.encode('ascii', 'ignore').decode('ascii')
    #text = emoji_pattern.sub(r'', text)
    
    text = re.sub('(http:\/\/www\.|https:\/\/www\.|http:\/\/|https:\/\/)?[a-z0-9]+([\-\.]{1}[a-z0-9]+)*\.[a-z]{2,5}(:[0-9]{1,5})?(\/.*)?\s',
                ' __url__ ',text) #remove urls
    #text = re.sub('\'','',text)
    #text = re.sub(r'\d+', ' __number__ ', text) #replaces numbers
    #text = re.sub('\W', ' ', text)
    text = re.sub('\[(.*?)\]','',text) #removes [this one]
    text = re.sub(' +', ' ', text)
    text = text.replace('\t', '')
    text = text.replace('\n', '')
    return text

In [20]:
positive_set = {}
train_pos_set = []
for i in range(len(paragraphs)):
    text_in_para= paragraphs[i]
    para = text_in_para.split('.')
    if (len(para)>(2*window+1)):
        sub_para = para[window+1:len(para)-(window+1)]
        cnt = 0
    
        while (cnt<= (len(para)-window)):
                seleted_text = random.choice(sub_para)
                index_train_sent1 = sub_para.index(seleted_text)
                index_train_sent2 = random.randint(index_train_sent1-window,index_train_sent1+window)
                train_sent1 = seleted_text
                train_sent2 = para[index_train_sent2]
                train_sent1 = cleanText(train_sent1)
                train_sent2 = cleanText(train_sent2)
                positive_set['sent_1']= train_sent1
                positive_set['sent_2']= train_sent2
                positive_set['label'] = 0  #if Next sentence label is 0
                cnt = cnt +1
                train_pos_set.append(positive_set.copy())
        else:
            continue

In [21]:
negative_set = {}
train_neg_set = []
for i in range(len(paragraphs)):
    text_in_para= paragraphs[i]
    para = text_in_para.split('.')
    r_i = random.randint(0,len(paragraphs)-1)
    if (r_i==i):
        r_i= r_i + 2 
    text_in_random_para = paragraphs[r_i]
    random_para = text_in_random_para.split('.')
    cnt =0
    while (cnt<= (len(para))):
        selected_text1 = random.choice(para)
        selected_text2 = random.choice(random_para)
        train_sent1 = selected_text1
        train_sent2 = selected_text2
        train_sent1 = cleanText(train_sent1)
        train_sent2 = cleanText(train_sent2)
        negative_set['sent_1']= train_sent1
        negative_set['sent_2']= train_sent2
        negative_set['label'] = 1  #if not Next sentence label is 1
        #print(negative_set)
        cnt = cnt +1
        train_neg_set.append(negative_set.copy())
        #print(cnt)
    else:
        continue

In [22]:
train_pos = pd.DataFrame(train_pos_set)
train_neg = pd.DataFrame(train_neg_set)

In [23]:
train_data = pd.concat([train_pos,train_neg])

In [24]:
train_data

,label,sent_1,sent_2
0,0,"So, let’s code these assumptions first:These s...","What can you do about it?Ideally, you’d like t..."
1,0,"To encode/decode a field, we need to know its ...",Armed with this knowledge we can now go back t...
2,0,As we’ve seen — names are encoded as literal t...,Armed with this knowledge we can now go back t...
3,0,We’ll need a type-class for this:If we were co...,"Now, what constitutes a supported type?The doc..."
4,0,"As far as they’re concerned, Claims is a type ...","So, let’s code these assumptions first:These s..."
5,0,"What can you do about it?Ideally, you’d like t...",This is a great library for JWT but when it co...
6,0,HList can be either:empty — represented by sin...,Armed with this knowledge we can now go back t...
7,0,ClaimsEncoder and ClaimsDecoder will be used a...,"As far as they’re concerned, Claims is a type ..."
8,0,That’s what we can use Shapeless for,ClaimsEncoder and ClaimsDecoder will be used a...
9,0,"To encode/decode a field, we need to know its ...",That’s what we can use Shapeless for


In [25]:
train_data = train_data.fillna(0)

In [26]:
train_data = train_data.sample(frac=1).reset_index(drop=True)

In [27]:
#train_data= train_data[['sent_1','sent_2','label']]
train_data.head()

,label,sent_1,sent_2
0,1,erdconfig file,
1,1,In case of a cache miss it forwards the reques...,This simple trick allows us to actually test o...
2,1,This is something I wish I’d done better earli...,Here’s the implementation of mapSearchPointCha...
3,1,"As we are coming from ES2, some types have cha...",js and Vuex (Medium)
4,0,It just did not get prioritized,"At times, it may be difficult to dig up the ri..."


In [28]:
df_full = train_data[train_data.sent_1.str.len()>30]
df_full = df_full[df_full.sent_2.str.len()>30].copy()

In [29]:
from pandas.api.types import is_numeric_dtype

In [30]:
is_numeric_dtype(df_full['label'])

True

In [31]:
len(df_full)

211896

In [32]:
train = df_full[0:int(len(df_full)*.8)]
val = df_full[int(len(df_full)*.8):int(len(df_full)*.8)+int(len(df_full)*.1)]
test = df_full[int(len(df_full)*.8)+int(len(df_full)*.1):]
len(train),len(val),len(test)

(169516, 21189, 21191)

In [33]:
train.to_csv('train_nsp.csv',sep="\t",index=True)
val.to_csv('val_nsp.csv',sep="\t",index=True)
test.to_csv('test_nsp.csv',sep="\t",index=True)

In [1]:
'''
# If data with single lable column
data = pd.read_csv('/home/shivam/Music/kaggle/toxic_comments_bert/data/toxic_comments/train_2.csv')
data_ = data
data_['target2']=data_['target'].apply(lambda x:1 if x==1 else 0)
data_['target']= data_['target'].apply(lambda x:1 if x==0 else 0)
dfx.to_csv('.csv',index=False)
'''

"\n# If data with single lable column\ndata = pd.read_csv('/home/shivam/Music/kaggle/toxic_comments_bert/data/toxic_comments/train_2.csv')\ndata_ = data\ndata_['target2']=data_['target'].apply(lambda x:1 if x==1 else 0)\ndata_['target']= data_['target'].apply(lambda x:1 if x==0 else 0)\ndfx.to_csv('.csv',index=False)\n"

### data processing was till here  Following is playground to play with the above data

## For NSP model EVAL call on above data    ## no training involved below

In [ ]:
class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, target):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.target = target

In [ ]:
def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""
    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()

In [ ]:
def convert_sentence_pair(titles, descs, max_seq_length, tokenizer):
    features = []
    for (ex_index, (title, desc)) in enumerate(zip(titles, descs)):
        tokens_a = tokenizer.tokenize(title)

        tokens_b = None
        tokens_b = tokenizer.tokenize(desc)
        # Modifies `tokens_a` and `tokens_b` in place so that the total
        # length is less than the specified length.
        # Account for [CLS], [SEP], [SEP] with "- 3"
        _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)

        # The convention in BERT is:
        # (a) For sequence pairs:
        #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
        #  type_ids: 0   0  0    0    0     0       0 0    1  1  1  1   1 1
        # (b) For single sequences:
        #  tokens:   [CLS] the dog is hairy . [SEP]
        #  type_ids: 0   0   0   0  0     0 0
        #
        # Where "type_ids" are used to indicate whether this is the first
        # sequence or the second sequence. The embedding vectors for `type=0` and
        # `type=1` were learned during pre-training and are added to the wordpiece
        # embedding vector (and position vector). This is not *strictly* necessary
        # since the [SEP] token unambigiously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.
        #
        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens += tokens_b + ["[SEP]"]
            segment_ids += [1] * (len(tokens_b) + 1)

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1] * len(input_ids)

        # Zero-pad up to the sequence length.
        padding = [0] * (max_seq_length - len(input_ids))
        input_ids += padding
        input_mask += padding
        segment_ids += padding

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length

        if ex_index < 5:
            logger.info("*** Example ***")
            logger.info("tokens: %s" % " ".join(
                    [str(x) for x in tokens]))
            logger.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
            logger.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
            logger.info(
                    "segment_ids: %s" % " ".join([str(x) for x in segment_ids]))

        features.append(
                InputFeatures(
                    input_ids=input_ids,
                    input_mask=input_mask,
                    segment_ids=segment_ids,
                    target=1
        ))
    return features

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True,cache_dir='/home/shivam/Documents/ether_language_model/bert_hugging_pytorch/custom/nsp_model')

In [ ]:
correct_pairs = convert_sentence_pair(df_full.sent_1.tolist(), df_full.sent_2.tolist(), max_seq_length=50, tokenizer=tokenizer)

In [ ]:
len(correct_pairs)

In [ ]:
model = BertForNextSentencePrediction.from_pretrained(
    "bert-base-uncased",
    cache_dir='/home/shivam/Documents/ether_language_model/bert_hugging_pytorch/custom/nsp_model'
).to(device)

In [ ]:
class BertForSequenceClassification(BertPreTrainedModel):      
    
        def __init__(self, config, num_labels):
            super(BertForSequenceClassification, self).__init__(config)
            self.num_labels = num_labels
            self.bert = BertModel(config)
            self.dropout = nn.Dropout(config.hidden_dropout_prob)
            self.classifier = nn.Linear(config.hidden_size, num_labels)
            self.apply(self.init_bert_weights)

        def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None):
            _, pooled_output = self.bert(input_ids, token_type_ids, attention_mask, output_all_encoded_layers=True)
            pooled_output = self.dropout(pooled_output)
            logits = self.classifier(pooled_output)

            if labels is not None:
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
                return loss
            else:
                return logits

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
config = BertConfig.from_json_file('/home/shivam/Documents/ether_language_model/bert_hugging_pytorch/custom/saved_models/bert_config.json')
model1 = BertForSequenceClassification(config,2)
m1_state_dict = torch.load('/home/shivam/Documents/ether_language_model/bert_hugging_pytorch/custom/saved_models/engg_sof/modelepc_10_lr_2e-05_seq_25_batch_64.bin',map_location={'cuda:0': 'cpu'})
model1.load_state_dict(m1_state_dict)